# Experiment: Comparing different devices for training

In [7]:
using Revise, QARBoM, DWave, DataFrames, CSV, QiskitOpt, MQLib, PySA

In [2]:
MOI = QARBoM.QUBO.ToQUBO.MOI
learning_rate = 0.0002
num_sweeps = [i for i in 1:25:101]
num_reads = [i for i in 1:25:101]
W = rand(22,10)

22×10 Matrix{Float64}:
 0.829383  0.99626    0.6818    …  0.0604107   0.356255    0.334519
 0.581794  0.898484   0.925647     0.0579176   0.891905    0.853225
 0.935916  0.58041    0.530307     0.282476    0.435834    0.143152
 0.899511  0.67061    0.293822     0.349012    0.886895    0.948902
 0.547355  0.179995   0.802362     0.749834    0.664147    0.631256
 0.670575  0.920065   0.984927  …  0.466139    0.708559    0.174579
 0.697038  0.900941   0.552187     0.195607    0.477171    0.467951
 0.719038  0.224267   0.606312     0.810888    0.269798    0.781863
 0.207038  0.610953   0.524908     0.317614    0.514316    0.03743
 0.808492  0.132892   0.157132     0.692356    0.798161    0.834695
 0.581777  0.574929   0.103696  …  0.617936    0.00637439  0.490381
 0.928209  0.493292   0.358249     0.548198    0.801672    0.946328
 0.191476  0.907377   0.426528     0.669111    0.175122    0.592911
 0.52079   0.0809426  0.748923     0.942057    0.889957    0.931211
 0.511923  0.300413   0.52

In [3]:
df = DataFrame(CSV.File(raw"./converted_bool_only.csv"))

x_train = Vector{Vector{Int}}()

for row in eachrow(df)
    push!(x_train, collect(row))
end

# DWave

In [17]:

rbm = QARBoM.QUBORBM(22,10, W, DWave.Neal.Optimizer)
MOI.supports(::DWave.Neal.Optimizer, ::MOI.ObjectiveSense) = true

MOI.set(rbm.model, MOI.RawOptimizerAttribute("num_reads"), 25)
MOI.set(rbm.model, MOI.RawOptimizerAttribute("num_sweeps"), 75)
mse_dwave = QARBoM.train_persistent_qubo(rbm, x_train[1:10000];n_samples = 1, batch_size = 10, n_epochs = 50, learning_rate = [learning_rate for i in 1:50])


Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |   15.8715 |        0.1631 |       6.3228 |        1.7566 |    8.2425 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |   11.8611 |        0.0274 |       4.6169 |        1.7738 |   14.6606 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | T

50-element Vector{Float64}:
 15.871472619124113
 11.861133943019048
  6.4191974064896264
  4.459284342335237
  4.032037764294216
  3.884924027979687
  3.8198648991142945
  3.7702576222592166
  3.7255209425817273
  3.7110388111996726
  3.6878999059953377
  3.665223361023916
  3.6449558219425118
  ⋮
  3.4064296831382315
  3.4040760676056183
  3.409913384147263
  3.4026329495808363
  3.3964520841933123
  3.385359973402523
  3.3700502721409022
  3.3581761243307673
  3.350049402076662
  3.342656928240713
  3.333622127523188
  3.3263707823591453

# Qiskit

In [8]:

rbm = QARBoM.QUBORBM(22,10, W, QiskitOpt.QAOA.Optimizer)
MOI.set(rbm.model, QAOA.NumberOfReads(), 1000)
MOI.set(rbm.model, QAOA.MaximumIterations(), 100)

MOI.set(rbm.model, QAOA.Channel(), "ibm_quantum") # or QAOA.Instance
MOI.set(rbm.model, QAOA.Instance(), "ibm-q-asu/main/purdue-david-ber") # or QAOA.Instance

MOI.set(rbm.model, QAOA.IBMBackend(), "ibm_kyiv") # or QAOA.IBMFakeBackend

mse_qiskit = QARBoM.train_persistent_qubo(rbm, x_train[1:10000];n_samples = 1, batch_size = 10, n_epochs = 50, learning_rate = [learning_rate for i in 1:50])


Setting mini-batches
Starting training
Running QAOA on ibm_kyiv...


LoadError: InterruptException:

# MQLib

In [6]:

rbm = QARBoM.QUBORBM(22,10, W, MQLib.Optimizer)
MOI.supports(::MQLib.Optimizer, ::MOI.ObjectiveSense) = true
QARBoM.JuMP.set_silent(rbm.model)


mse_mqlib = QARBoM.train_persistent_qubo(rbm, x_train[1:10000];n_samples = 1, batch_size = 10, n_epochs = 50, learning_rate = [learning_rate for i in 1:50])


Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |   16.8539 |        0.0144 |       9.0130 |        1.3254 |   10.3528 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |   17.0139 |        0.0244 |       8.2920 |        1.2866 |   19.9557 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | T

┌ Error: mktempdir cleanup
│   exception =
│    InterruptException:
│    Stacktrace:
│      [1] rethrow()
│        @ Base ./error.jl:61
│      [2] wait(c::Base.GenericCondition{ReentrantLock}; first::Bool)
│        @ Base ./condition.jl:133
│      [3] wait
│        @ ./condition.jl:125 [inlined]
│      [4] take_unbuffered(c::Channel{Tuple{String, Vector{String}, Vector{String}}})
│        @ Base ./channels.jl:494
│      [5] take!
│        @ ./channels.jl:471 [inlined]
│      [6] iterate(c::Channel{Tuple{String, Vector{String}, Vector{String}}}, state::Nothing)
│        @ Base ./channels.jl:613
│      [7] prepare_for_deletion(path::String)
│        @ Base.Filesystem ./file.jl:515
│      [8] mktempdir(fn::MQLib.var"#3#5"{Float64, Dict{String, Any}, Vector{QUBOTools.Sample{Float64, Int64}}, Float64, Nothing, Bool, Int64, QUBOTools.Model{Int64, Float64, Int64, QUBOTools.Form{Float64, QUBOTools.SparseLinearForm{Float64}, QUBOTools.SparseQuadraticForm{Float64}}}, Float64, Float64}, parent::S

|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     9 |   17.2548 |        0.0244 |       9.4227 |        1.4173 |   89.9367 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|    10 |   17.2635 |        0.0143 |       9.7179 |        1.5204 |  101.1892 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|---------------------------

LoadError: InterruptException:

# PySA

In [9]:

rbm = QARBoM.QUBORBM(22,10, W, PySA.Optimizer)
MOI.supports(::PySA.Optimizer, ::MOI.ObjectiveSense) = true
QARBoM.JuMP.set_silent(rbm.model)

MOI.set(rbm.model, PySA.NumberOfSweeps(), 5)


mse_pysa = QARBoM.train_persistent_qubo(rbm, x_train[1:10000];n_samples = 1, batch_size = 10, n_epochs = 50, learning_rate = [learning_rate for i in 1:50])


Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |   16.1086 |        0.0155 |      24.2550 |        1.5632 |   25.8336 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |   13.7419 |        0.0161 |      23.6589 |        1.5599 |   51.0684 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | T

50-element Vector{Float64}:
 16.108628274052464
 13.741947030471692
 10.136013848900012
  6.944069030468493
  5.0317664679375325
  4.057801509124256
  3.5437565123284482
  3.2599304643545715
  3.0964631400571125
  2.9903009101920706
  2.9121831877721664
  2.855490386149155
  2.812577851377279
  ⋮
  2.7545793464959294
  2.755066378974357
  2.7554376482412963
  2.7552573712186756
  2.756063732730058
  2.7564371174519082
  2.7559184277999242
  2.755969002523728
  2.755797462739747
  2.75633621642235
  2.7559940937760685
  2.755582216612159

In [13]:
df_pysa = DataFrame(:mse => mse_pysa)
CSV.write("mse_pysa.csv", df_pysa)

"mse_pysa.csv"